# Virtual Image Diffusion Transformer Micro Server
---


##  Setup all lib and env:
**Python Lib:**
>* FastApi - *for client access* 
>* redis - *for base data*
>* websockets - *for desktop control* 
>* aiortc - *for desktop streaming*

```
pip install fastapi
pip install uvicorn
pip install transformers
pip install torch
pip install redis
pip install aioredis
pip install python-socketio[client]
pip install aiortc
pip install av
pip install aiohttp


```

In [1]:
# Setup all lib and env

# fastapi :
try:
    import fastapi                      
    print('fastapi: already installed')
except ImportError:
  !python -m pip install -q fastapi
  print('Installed fastapi')
    
# uvicorn :
try:
    import uvicorn                      
    print('uvicorn: already installed')
except ImportError:
  !python -m pip install -q uvicorn
  print('Installed uvicorn')
    
# transformers :
try:
    import transformers                      
    print('transformers: already installed')
except ImportError:
  !python -m pip install -q transformers
  print('Installed transformers')
    
# torch :
try:
    import torch                      
    print('torch: already installed')
except ImportError:
  !python -m pip install -q torch
  print('Installed torch')
    
# redis :
try:
    import redis                      
    print('redis: already installed')
except ImportError:
  !python -m pip install -q redis
  print('Installed redis')
    
# aioredis :
try:
    import aioredis                      
    print('aioredis: already installed')
except ImportError:
  !python -m pip install -q aioredis
  print('Installed aioredis')
    
# python-socketio[client] :
try:
    import socketio                      
    print('python-socketio[client]: already installed')
except ImportError:
  !python -m pip install -q python-socketio[client]
  print('Installed socketio')
    
# aiortc :
try:
    import aiortc                      
    print('aiortc: already installed')
except ImportError:
  !python -m pip install -q aiortc
  print('Installed aiortc')
    
# av :
try:
    import av                      
    print('av: already installed')
except ImportError:
  !python -m pip install -q av
  print('Installed av')
    
# aiohttp :
try:
    import aiohttp                      
    print('aiohttp: already installed')
except ImportError:
  !python -m pip install -q aiohttp
  print('Installed aiohttp')


fastapi: already installed
uvicorn: already installed
transformers: already installed
torch: already installed
redis: already installed
aioredis: already installed
python-socketio[client]: already installed
aiortc: already installed
av: already installed
aiohttp: already installed


---
## Setup Language Trasnformer Stream API:

## Import libraries:

In [2]:
import json
import asyncio
from fastapi import FastAPI, WebSocket, Request
from transformers import CLIPProcessor, CLIPModel
from pydantic import BaseModel
import redis
import aioredis
import socketio
from aiortc import RTCPeerConnection, RTCSessionDescription
from aiortc.contrib.media import MediaBlackhole


---
## Define the FastAPI app, Redis configuration, and CLIP model:

In [3]:
app = FastAPI()

# Redis configuration
REDIS_URL = "redis://localhost:6379"
redis_client = redis.Redis.from_url(REDIS_URL)
redis_pubsub = redis_client.pubsub()

# Socket.IO client
sio = socketio.AsyncClient()

---
## Load the pre-trained model and tokenizer:
**Choose a pre-trained model:**

Select a suitable pre-trained model from the Hugging Face Model Hub [openai/clip](https://openai/clip-vit-base-patch32). 
> For example, you can use "openai/clip-vit-base-patch32".


## Prepare your dataset:

> Transform your dataset into a format suitable for training. You'll need to tokenize your text data and create input sequences. You can use the GPT2Tokenizer for this purpose.

In [4]:
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

---

## Fine-tune the model:

>Fine-tune the pre-trained model on your dataset using a suitable training loop. You can use the Trainer class from the

---

---

## Create a Pydantic model to accept input:

In [5]:
class ImageTextComparisonInput(BaseModel):
    image_url: str
    text: str

---
## Define the FastAPI endpoint for  image-text comparison:

In [6]:
import requests
from PIL import Image
from io import BytesIO

@app.post("/compare")
async def compare_image_text(input_data: ImageTextComparisonInput):
    image_url = input_data.image_url
    text = input_data.text

    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    
    inputs = clip_processor(text=text, images=image, return_tensors="pt", padding=True)
    logits_per_image = clip_model(**inputs).logits_per_image
    probs = logits_per_image.softmax(dim=1).tolist()

    # Log interaction to Redis
    interaction_data = {
        "image_url": image_url,
        "text": text,
        "comparison_score": probs,
    }
    redis_client.set("interaction", json.dumps(interaction_data))

    # Publish event to Redis Pub/Sub
    redis_client.publish("events", json.dumps(interaction_data))

    return {"comparison_score": probs}


---
## Implement WebRTC for real-time video processing:

In [7]:
class RTCConnection:
    def __init__(self):
        self.peer_connection = RTCPeerConnection()

    async def set_description(self, sdp):
        await self.peer_connection.setRemoteDescription(RTCSessionDescription(**sdp))
        await self.peer_connection.setLocalDescription(await self.peer_connection.createAnswer())

        return self.peer_connection.localDescription

    async def close(self):
        await self.peer_connection.close()

connections = {}

@app.post("/offer")
async def offer(request: Request):
    params = await request.json()
    connection_id = params["connectionId"]
    sdp = params["sdp"]

    connection = RTCConnection()
    local_description = await connection.set_description(sdp)

    connections[connection_id] = connection

    return {"sdp": local_description.sdp, "type": local_description.type}

@app.post("/close/{connection_id}")
async def close(connection_id: str):
    if connection_id in connections:
        await connections[connection_id].close()
        del connections[connection_id]

    return {"result": "connection_closed"}


---
## Run the FastAPI application in the Jupyter Notebook:

> Note that the pyngrok package is optional and only needed if you want to expose your API to the internet using ngrok. To install the pyngrok package, run:

In [8]:
#pip install pyngrok
# pyngrok :
try:
    import pyngrok                      
    print('pyngrok: already installed')
except ImportError:
  !python -m pip install -q pyngrok
  print('Installed pyngrok')

pyngrok: already installed


In [9]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

# Set up ngrok for external access (optional)
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8400)


t=2023-08-07T22:11:56-0700 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\wesle\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\wesle\\.ngrok2\\ngrok.yml
INFO:     Started server process [151004]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8400 (Press CTRL+C to quit)


Public URL: https://50bf-69-172-154-170.ngrok.io


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [151004]


---
## Test API with a gardio interface

In [10]:
# # pip install gradio

# # gradio :
# try:
#     import gradio                      
#     print('gradio: already installed')
# except ImportError:
#   !python -m pip install -q gradio
#   print('Installed gradio')


In [11]:
# import gradio as gr

# def compare_image_text(image, text):
#     import requests
#     from PIL import Image
#     import io

#     buffer = io.BytesIO()
#     image.save(buffer, format="PNG")
#     buffer.seek(0)

#     image_url = "your_image_upload_service_url"  # Replace with a service to upload images and return their URL
#     response = requests.post(image_url, files={"image": buffer})

#     if response.status_code == 200:
#         image_url = response.json()["url"]
#         response = requests.post("http://localhost:8000/compare", json={"image_url": image_url, "text": text})
#         if response.status_code == 200:
#             comparison_score = response.json()["comparison_score"]
#             return {"Comparison Score": comparison_score}
#         else:
#             return {"Comparison Score": "Error"}
#     else:
#         return {"Comparison Score": "Error"}

# iface = gr.Interface(fn=compare_image_text, inputs=["image", "text"], outputs="text", title="CLIP Image-Text Comparison")
# iface.launch()
